In [1]:
!pip install transformers

     |████████████████████████████████| 2.1MB 16.2MB/s 
     |████████████████████████████████| 3.3MB 52.2MB/s 
     |████████████████████████████████| 901kB 40.3MB/s 


In [2]:
 CUDA_LAUNCH_BLOCKING=1 

In [3]:
!pip install seqeval

     |████████████████████████████████| 51kB 7.2MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16172 sha256=5280c4f2bc5601559ecda7a09d32809e8de76ae8ce960dfa6a550d7abc35169f
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval


In [4]:
import pandas as pd
import math
import numpy as np
from seqeval.metrics import f1_score
from seqeval.metrics import classification_report,accuracy_score,f1_score
import torch.nn.functional as F

In [5]:
import torch
import os
from tqdm import tqdm,trange
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig
from transformers import BertForTokenClassification, AdamW

In [6]:
#!pip install pandas==0.22

In [7]:
# Check library version
!pip list | grep -E 'transformers|torch|Keras'

Keras                         2.4.3         
Keras-Preprocessing           1.1.2         
torch                         1.8.1+cu101   
torchsummary                  1.5.1         
torchtext                     0.9.1         
torchvision                   0.9.1+cu101   
transformers                  4.5.1         


In [8]:
#df= pd.read_csv('medicare_BOI_train.csv')

In [9]:
# Fillna method can make same sentence with same sentence name
#df_data = pd.read_csv('train_dataset_BOI_new1.csv',sep=",",encoding="latin1").fillna(method='ffill')
#df_data = pd.read_csv('sample_data/sample_input_dataset_bot.csv',sep=",",encoding="latin1").fillna(method='ffill')
df_data = pd.read_csv('sample_data/medicare_train_dataset_BOI_input_manual_v3.csv',sep=",",encoding="latin1").fillna(method='ffill')
#df_data = pd.read_csv('medicare_train_dataset_BOI_input.csv',sep=",",encoding="latin1").fillna(method='ffill')

In [10]:
df_data.columns

Index(['sentence #', 'pattern', 'benefit_type', 'requirement_text', 'word',
       'POS', 'tags'],
      dtype='object')

In [11]:
df_data.head(n=20)

,sentence #,pattern,benefit_type,requirement_text,word,POS,tags
0,sentence 1,"Includes programs such as group exercise, adul...",ADCS,Adult Day Care Services,Includes,NNS,O
1,sentence 1,"Includes programs such as group exercise, adul...",ADCS,Adult Day Care Services,programs,NNS,O
2,sentence 1,"Includes programs such as group exercise, adul...",ADCS,Adult Day Care Services,such,JJ,O
3,sentence 1,"Includes programs such as group exercise, adul...",ADCS,Adult Day Care Services,as,IN,O
4,sentence 1,"Includes programs such as group exercise, adul...",ADCS,Adult Day Care Services,group,NN,B_INCLUSION
5,sentence 1,"Includes programs such as group exercise, adul...",ADCS,Adult Day Care Services,exercise,NN,I_INCLUSION
6,sentence 1,"Includes programs such as group exercise, adul...",ADCS,Adult Day Care Services,adult,NN,B_INCLUSION
7,sentence 1,"Includes programs such as group exercise, adul...",ADCS,Adult Day Care Services,education,NN,I_INCLUSION
8,sentence 1,"Includes programs such as group exercise, adul...",ADCS,Adult Day Care Services,classes,NNS,I_INCLUSION
9,sentence 1,"Includes programs such as group exercise, adul...",ADCS,Adult Day Care Services,recreation,NN,U_INCLUSION


In [12]:
df_data.POS.unique()

array(['NNS', 'JJ', 'IN', 'NN', 'CC', 'VBZ', 'TO', 'VB', 'PRP', 'MD',
       'RB', 'DT', 'VBN', 'WDT', 'VBG', 'NNP', 'RBR', 'VBP', ':', 'NNPS',
       'CD', 'VBD', 'EX', 'PRP$', 'JJS', 'SYM', 'WRB', 'WP', 'JJR', 'RP',
       'WP$', 'RBS'], dtype=object)

In [13]:
df_data.tags.unique()

array(['O', 'B_INCLUSION', 'I_INCLUSION', 'U_INCLUSION', 'B_EXCLUSION',
       'I_EXCLUSION', 'U_EXCLUSION', 'i_EXCLUSION'], dtype=object)

In [14]:
# Analyse summary of data
df_data['sentence #'].nunique(), df_data.word.nunique(), df_data.POS.nunique(), df_data.tags.nunique()

(250, 1677, 32, 8)

In [15]:
# Analyse the Tag distribution
df_data.tags.value_counts()

O              4874
I_INCLUSION     869
B_INCLUSION     391
I_EXCLUSION     238
U_INCLUSION     121
B_EXCLUSION      92
U_EXCLUSION      31
i_EXCLUSION       6
Name: tags, dtype: int64

In [16]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w,b,r, p, t) for w,b,r, p, t in zip(s['word'].values.tolist(),
                                                           s['benefit_type'].values.tolist(),
                                                           s['requirement_text'].values.tolist(),
                                                           s['POS'].values.tolist(), 
                                                           s['tags'].values.tolist())]
        self.grouped = self.data.groupby('sentence #').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None

In [17]:
# Get full document data struce
getter = SentenceGetter(df_data)

In [18]:
# Get sentence data
sentences = [[s[0] for s in sent] for sent in getter.sentences]
sentences[0]

['Includes',
 'programs',
 'such',
 'as',
 'group',
 'exercise',
 'adult',
 'education',
 'classes',
 'recreation',
 'nutritious',
 'meals',
 'and',
 'social',
 'work',
 'services']

In [19]:
# Get pos data
poses = [[s[3] for s in sent] for sent in getter.sentences]
print(poses[0])

['NNS', 'NNS', 'JJ', 'IN', 'NN', 'NN', 'NN', 'NN', 'NNS', 'NN', 'JJ', 'NNS', 'CC', 'JJ', 'NN', 'NNS']


In [20]:
# Get tag labels data
labels = [[s[4] for s in sent] for sent in getter.sentences]
print(labels[0])

['O', 'O', 'O', 'O', 'B_INCLUSION', 'I_INCLUSION', 'B_INCLUSION', 'I_INCLUSION', 'I_INCLUSION', 'U_INCLUSION', 'B_INCLUSION', 'I_INCLUSION', 'O', 'B_INCLUSION', 'I_INCLUSION', 'I_INCLUSION']


In [21]:
tags_vals = list(set(df_data["tags"].values))

In [22]:
# Add X  label for word piece support
# Add [CLS] and [SEP] as BERT need
tags_vals.append('X')
tags_vals.append('[CLS]')
tags_vals.append('[SEP]')

In [23]:
tags_vals = set(tags_vals)

In [24]:
tags_vals

{'B_EXCLUSION',
 'B_INCLUSION',
 'I_EXCLUSION',
 'I_INCLUSION',
 'O',
 'U_EXCLUSION',
 'U_INCLUSION',
 'X',
 '[CLS]',
 '[SEP]',
 'i_EXCLUSION'}

In [25]:
# Set a dict for mapping id to tag name
#tag2idx = {t: i for i, t in enumerate(tags_vals)}
# Recommend to set it by manual define, good for reusing
# tag2idx={'B_EXCLUSION': 14,
#  'B_INCLUSION': 2,
#  'I_EXCLUSION': 0,
#  'I_INCLUSION': 3,
#  'O': 4,
#  'X': 5,
#  '[CLS]':6,
#  '[SEP]':7}

In [26]:
tag2idx={'B_INCLUSION': 0,
 'U_INCLUSION': 1,
 'I_INCLUSION': 2,
 'i_EXCLUSION': 3,
 'O': 4,
 'U_EXCLUSION': 5,
 'I_EXCLUSION': 6,
 'B_EXCLUSION': 7}

In [27]:
tag2idx

{'B_EXCLUSION': 7,
 'B_INCLUSION': 0,
 'I_EXCLUSION': 6,
 'I_INCLUSION': 2,
 'O': 4,
 'U_EXCLUSION': 5,
 'U_INCLUSION': 1,
 'i_EXCLUSION': 3}

In [28]:
# Mapping index to name
tag2name={tag2idx[key] : key for key in tag2idx.keys()}

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [30]:
n_gpu

1

In [31]:
# Manual define vocabulary address, if you download the tokenzier file in local
# vocab.txt, download from: https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt
#vocabulary = "models/bert-base-cased/vocab.txt"

In [32]:
# Len of the sentence must be not bigger than the training model
# See model's 'max_position_embeddings' = 512
max_len  = 45

In [33]:
!pip install pytorch-pretrained-bert

     |████████████████████████████████| 133kB 18.5MB/s 
     |████████████████████████████████| 133kB 34.5MB/s 
     |████████████████████████████████| 7.5MB 42.1MB/s 
     |████████████████████████████████| 81kB 11.9MB/s 
ERROR: botocore 1.20.62 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [34]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

In [35]:
# load tokenizer, with manual file address or pretrained address
#tokenizer=BertTokenizer(vocab_file=vocabulary,do_lower_case=False)

In [36]:
# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

INFO:pytorch_pretrained_bert.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt not found in cache, downloading to /tmp/tmph61ozod2
100%|██████████| 213450/213450 [00:00<00:00, 23789604.57B/s]
INFO:pytorch_pretrained_bert.file_utils:copying /tmp/tmph61ozod2 to cache at /root/.pytorch_pretrained_bert/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
INFO:pytorch_pretrained_bert.file_utils:creating metadata file for /root/.pytorch_pretrained_bert/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
INFO:pytorch_pretrained_bert.file_utils:removing temp file /tmp/tmph61ozod2
INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at /root/.pytorch_pretrained_bert/5e8a2b4893d13790ed4150ca1906be5f7a03d6

In [37]:
tokenized_texts = []
word_piece_labels = []
i_inc = 0
for word_list,label in (zip(sentences,labels)):
    temp_lable = []
    temp_token = []
    
    # Add [CLS] at the front 
    temp_lable.append('[CLS]')
    temp_token.append('[CLS]')
    
    for word,lab in zip(word_list,label):
        token_list = tokenizer.tokenize(word)
        for m,token in enumerate(token_list):
            temp_token.append(token)
            if m==0:
                temp_lable.append(lab)
            else:
                temp_lable.append('X')  
                
    # Add [SEP] at the end
    temp_lable.append('[SEP]')
    temp_token.append('[SEP]')
    
    tokenized_texts.append(temp_token)
    word_piece_labels.append(temp_lable)
    
    if 5 > i_inc:
        print("No.%d,len:%d"%(i_inc,len(temp_token)))
        print("texts:%s"%(" ".join(temp_token)))
        print("No.%d,len:%d"%(i_inc,len(temp_lable)))
        print("lables:%s"%(" ".join(temp_lable)))
    i_inc +=1
    

No.0,len:20
texts:[CLS] Includes programs such as group exercise adult education classes recreation nut ##rit ##ious meals and social work services [SEP]
No.0,len:20
lables:[CLS] O O O O B_INCLUSION I_INCLUSION B_INCLUSION I_INCLUSION I_INCLUSION U_INCLUSION B_INCLUSION X X I_INCLUSION O B_INCLUSION I_INCLUSION I_INCLUSION [SEP]
No.1,len:21
texts:[CLS] 4 ) Op ##io ##id use disorder treatment services are covered under Part B of Original Me ##dic ##are [SEP]
No.1,len:21
lables:[CLS] O X B_INCLUSION X X I_INCLUSION I_INCLUSION I_INCLUSION O O O O O X O O O X X [SEP]
No.2,len:39
texts:[CLS] Op ##tional Su ##pp ##lement ##al Pack ##age 2 - Dental and Vision Pack ##age covers prevent ##ive rest ##ora ##tive end ##od ##ont ##ic period ##ont ##ic and oral surgery dental services and vision eye ##wear [SEP]
No.2,len:39
lables:[CLS] O X O X X X O X X O O O O O X O U_INCLUSION X U_INCLUSION X X U_INCLUSION X X X U_INCLUSION X X O B_INCLUSION I_INCLUSION I_INCLUSION I_INCLUSION O B_INCLUSION I_IN

In [38]:
# Make text token into id
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=max_len, dtype="long", truncating="post", padding="post")
print(input_ids[0])

[  101 25051  2648  1216  1112  1372  6730  4457  1972  3553 12729 22664
  7729  4179 13077  1105  1934  1250  1826   102     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0]


In [39]:
# Make label into id, pad with "O" meaning others
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in word_piece_labels],
                     maxlen=max_len, value=tag2idx["O"], padding="post",
                     dtype="float", truncating="post")
print(tags[0])

[nan  4.  4.  4.  4.  0.  2.  0.  2.  2.  1.  0. nan nan  2.  4.  0.  2.
  2. nan  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.
  4.  4.  4.  4.  4.  4.  4.  4.  4.]


In [40]:
tags[np.isnan(tags)] = 0

In [41]:
print(tags[0])

[0. 4. 4. 4. 4. 0. 2. 0. 2. 2. 1. 0. 0. 0. 2. 4. 0. 2. 2. 0. 4. 4. 4. 4.
 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.]


In [42]:
# For fine tune of predict, with token mask is 1,pad token is 0
attention_masks = [[int(i>0) for i in ii] for ii in input_ids]
attention_masks[0];

In [43]:
# Since only one sentence, all the segment set to 0
segment_ids = [[0] * len(input_id) for input_id in input_ids]
segment_ids[0];

In [44]:
tr_inputs, val_inputs, tr_tags, val_tags,tr_masks, val_masks,tr_segs, val_segs = train_test_split(input_ids, tags,attention_masks,segment_ids, 
                                                            random_state=4, test_size=0.2)

In [45]:
len(tr_inputs),len(val_inputs),len(tr_segs),len(val_segs)

(200, 50, 200, 50)

In [46]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)
tr_segs = torch.tensor(tr_segs)
val_segs = torch.tensor(val_segs)

In [47]:
# Set batch num
batch_num = 1

In [48]:
# Only set token embedding, attention embedding, no segment embedding
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
# Drop last can make batch training better for the last one
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_num,drop_last=True)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_num)

In [49]:
# #train_data = torch.tensor(train_data)
# #validation_inputs = torch.tensor(validation_inputs)
# train_labels = torch.tensor(train_labels, dtype=torch.long)
# validation_labels = torch.tensor(validation_labels, dtype=torch.long)
# train_masks = torch.tensor(train_masks, dtype=torch.long)
# validation_masks = torch.tensor(validation_masks, dtype=torch.long)

In [50]:
# In this folder, contain model confg(json) and model weight(bin) files
# pytorch_model.bin, download from: https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-pytorch_model.bin
# config.json, downlaod from: https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json
#model_file_address = 'models/bert-base-cased'

In [51]:
len(tag2idx)

8

In [52]:
# Will load config and weight with from_pretrained()
#model = BertForTokenClassification.from_pretrained(model_file_address,num_labels=len(tag2idx))
model = BertForTokenClassification.from_pretrained("bert-base-cased", num_labels=len(tag2idx))

INFO:filelock:Lock 140650088181584 acquired on /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307.lock


INFO:filelock:Lock 140650088181584 released on /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307.lock
INFO:filelock:Lock 140650079003984 acquired on /root/.cache/huggingface/transformers/092cc582560fc3833e556b3f833695c26343cb54b7e88cd02d40821462a74999.1f48cab6c959fc6c360d22bea39d06959e90f5b002e77e836d2da45464875cda.lock


INFO:filelock:Lock 140650079003984 released on /root/.cache/huggingface/transformers/092cc582560fc3833e556b3f833695c26343cb54b7e88cd02d40821462a74999.1f48cab6c959fc6c360d22bea39d06959e90f5b002e77e836d2da45464875cda.lock


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [53]:
model;

In [54]:
# Set model to GPU,if you are using GPU machine
model.cuda();

In [55]:
# Add multi GPU support
if n_gpu >1:
    model = torch.nn.DataParallel(model)

In [56]:
# Set epoch and grad max num
epochs = 2
max_grad_norm = 1.0

In [57]:
# Cacluate train optimiazaion num
num_train_optimization_steps = int( math.ceil(len(tr_inputs) / batch_num) / 1) * epochs

In [58]:
# True: fine tuning all the layers 
# False: only fine tuning the classifier layers
FULL_FINETUNING = True

In [59]:
if FULL_FINETUNING:
    # Fine tune model all layer parameters
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    # Only fine tune classifier parameters
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5)

In [60]:
# TRAIN loop
model.train();

In [61]:
print("***** Running training *****")
print("  Num examples = %d"%(len(tr_inputs)))
print("  Batch size = %d"%(batch_num))
print("  Num steps = %d"%(num_train_optimization_steps))
for _ in trange(epochs,desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
          ###############Bug fix code####################
        b_input_ids = b_input_ids.type(torch.LongTensor)
        b_input_mask = b_input_mask.type(torch.LongTensor)
        b_labels = b_labels.type(torch.LongTensor)

        b_input_ids = b_input_ids.to(device)
        b_input_mask = b_input_mask.to(device)
        b_labels = b_labels.to(device)
        
        # forward pass
        outputs = model(b_input_ids, token_type_ids=None,
        attention_mask=b_input_mask, labels=b_labels)
        loss, scores = outputs[:2]
        if n_gpu>1:
            # When multi gpu, average it
            loss = loss.mean()
        
        # backward pass
        loss.backward()
        
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        
        # update parameters
        optimizer.step()
        optimizer.zero_grad()
        
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
        

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 200
  Batch size = 1
  Num steps = 400


Epoch:  50%|█████     | 1/2 [00:13<00:13, 13.91s/it]

Train loss: 0.7629079227522015


Epoch: 100%|██████████| 2/2 [00:27<00:00, 13.73s/it]

Train loss: 0.3871274642483331


In [62]:
bert_out_address = 'sample_data/models/bert_out_model/en09'

In [63]:
# Make dir if not exits
if not os.path.exists(bert_out_address):
        os.makedirs(bert_out_address)

In [64]:
# Save a trained model, configuration and tokenizer
model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self

In [65]:
# If we save using the predefined names, we can load using `from_pretrained`
output_model_file = os.path.join(bert_out_address, "pytorch_model.bin")
output_config_file = os.path.join(bert_out_address, "config.json")

In [66]:
# Save model into file
torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(bert_out_address)

'sample_data/models/bert_out_model/en09/vocab.txt'

In [67]:
model = BertForTokenClassification.from_pretrained(bert_out_address,num_labels=len(tag2idx))

In [68]:
# Set model to GPU
model.cuda();

In [69]:
if n_gpu >1:
    model = torch.nn.DataParallel(model)

In [70]:
# Evalue loop
model.eval();

In [71]:
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
y_true = []
y_pred = []

print("***** Running evaluation *****")
print("  Num examples ={}".format(len(val_inputs)))
print("  Batch size = {}".format(batch_num))
for step, batch in enumerate(valid_dataloader):
    batch = tuple(t.to(device) for t in batch)
    input_ids, input_mask, label_ids = batch
      ###############Bug fix code####################
#     input_ids = input_ids.type(torch.LongTensor)
#     input_mask = input_mask.type(torch.LongTensor)
#     label_ids = label_ids.type(torch.LongTensor)

#     input_ids = input_ids.to(device)
#     input_mask = input_mask.to(device)
#     label_ids = label_ids.to(device)
    
#     input_ids = torch.where(torch.isnan(input_ids), torch.zeros_like(input_ids), input_ids)
#     input_mask = torch.where(torch.isnan(input_mask), torch.zeros_like(input_mask), input_mask)
#     label_ids = torch.where(torch.isnan(label_ids), torch.zeros_like(label_ids), label_ids)
    #print(input_ids)
    
#     if step > 2:
#         break
    
    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=None,
        attention_mask=input_mask,)
        # For eval mode, the first result of outputs is logits
        logits = outputs[0] 
    
    # Get NER predict result
    logits = torch.argmax(F.log_softmax(logits,dim=2),dim=2)
    logits = logits.detach().cpu().numpy()
    
    
    # Get NER true result
    label_ids = label_ids.to('cpu').numpy()
    
    
    # Only predict the real word, mark=0, will not calculate
    input_mask = input_mask.to('cpu').numpy()
    
    # Compare the valuable predict result
    for i,mask in enumerate(input_mask):
        # Real one
        temp_1 = []
        # Predict one
        temp_2 = []
        
        for j, m in enumerate(mask):
            # Mark=0, meaning its a pad word, dont compare
            if m:
                if tag2name[label_ids[i][j]] != "X" and tag2name[label_ids[i][j]] != "[CLS]" and tag2name[label_ids[i][j]] != "[SEP]" : # Exclude the X label
                    temp_1.append(tag2name[label_ids[i][j]])
                    temp_2.append(tag2name[logits[i][j]])
            else:
                break
        
            
        y_true.append(temp_1)
        y_pred.append(temp_2)

        

print("f1 socre: %f"%(f1_score(y_true, y_pred)))
print(y_true[0])
print(y_pred[0])
print("Accuracy score: %f"%(accuracy_score(y_true, y_pred)))

# Get acc , recall, F1 result report
report = classification_report(y_true, y_pred,digits=4)

# Save the report into file
output_eval_file = os.path.join(bert_out_address, "eval_results.txt")
with open(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    print("\n%s"%(report))
    print("f1 socre: %f"%(f1_score(y_true, y_pred)))
    print("Accuracy score: %f"%(accuracy_score(y_true, y_pred)))
    
    writer.write("f1 socre:\n")
    writer.write(str(f1_score(y_true, y_pred)))
    writer.write("\n\nAccuracy score:\n")
    writer.write(str(accuracy_score(y_true, y_pred)))
    writer.write("\n\n")  
    writer.write(report)

***** Running evaluation *****
  Num examples =50
  Batch size = 1
f1 socre: 0.781427
['B_INCLUSION', 'O', 'B_INCLUSION', 'B_INCLUSION', 'B_INCLUSION', 'B_INCLUSION', 'I_INCLUSION', 'I_INCLUSION', 'B_INCLUSION', 'O', 'O', 'O', 'O', 'O', 'B_INCLUSION', 'O', 'O', 'O', 'O', 'O', 'B_INCLUSION', 'B_INCLUSION', 'B_INCLUSION', 'O', 'B_INCLUSION', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B_INCLUSION']
['B_INCLUSION', 'O', 'B_INCLUSION', 'B_INCLUSION', 'B_INCLUSION', 'B_INCLUSION', 'I_INCLUSION', 'I_INCLUSION', 'B_INCLUSION', 'O', 'O', 'O', 'O', 'O', 'B_INCLUSION', 'O', 'O', 'O', 'O', 'O', 'B_INCLUSION', 'B_INCLUSION', 'B_INCLUSION', 'O', 'B_INCLUSION', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B_INCLUSION']
Accuracy score: 0.843794
***** Eval results *****

              precision    recall  f1-score   support

  _EXCLUSION     0.5385    0.2333    0.3256        30
  _INCLUSION     0.7916    0.8184    0.8048       413

   micro avg     0.7841    0.7788    0.7814       443
   macro avg 

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_INCLUSION seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: I_INCLUSION seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: U_INCLUSION seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_EXCLUSION seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: I_EXCLUSION seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/met

In [72]:
# Model we trained before, the dir containing pytorch_model.bin and vocab.txt
save_model_address = 'sample_data/models/bert_out_model/en09'

In [73]:
save_model = BertForTokenClassification.from_pretrained(save_model_address,num_labels=len(tag2idx))

In [74]:
# Here, our save model address containing pytorch_model.bin and vocab.txt
# So, we can load the tokenzier from the same dir as the save model address
tokenizer = BertTokenizer.from_pretrained(save_model_address,do_lower_case=False)

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file sample_data/models/bert_out_model/en09/vocab.txt


In [75]:
# Set max sentence length, must be the same as our training process
max_len  = 45

In [76]:
test_query = "Includes programs such as group exercise, adult education classes, recreation, nutritious meals, and social work services."

In [77]:
tokenized_texts = []

In [78]:
temp_token = []

In [79]:
# Add [CLS] at the front 
temp_token.append('[CLS]')

In [80]:
token_list = tokenizer.tokenize(test_query)

In [81]:
#token_list

In [82]:
for m,token in enumerate(token_list):
    temp_token.append(token)

In [83]:
# Trim the token to fit the length requirement
if len(temp_token) > max_len-1:
    temp_token= temp_token[:max_len-1]

In [84]:
# Add [SEP] at the end
temp_token.append('[SEP]')

In [85]:
temp_token

['[CLS]',
 'Includes',
 'programs',
 'such',
 'as',
 'group',
 'exercise',
 ',',
 'adult',
 'education',
 'classes',
 ',',
 'recreation',
 ',',
 'nut',
 '##rit',
 '##ious',
 'meals',
 ',',
 'and',
 'social',
 'work',
 'services',
 '.',
 '[SEP]']

In [86]:

tokenized_texts.append(temp_token)

In [87]:
# Make text token into id
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=max_len, dtype="long", truncating="post", padding="post")
print(input_ids[0])

[  101 25051  2648  1216  1112  1372  6730   117  4457  1972  3553   117
 12729   117 22664  7729  4179 13077   117  1105  1934  1250  1826   119
   102     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0]


In [88]:
# For fine tune of predict, with token mask is 1,pad token is 0
attention_masks = [[int(i>0) for i in ii] for ii in input_ids]
attention_masks[0];

In [89]:
segment_ids = [[0] * len(input_id) for input_id in input_ids]
segment_ids[0];

In [90]:
input_ids = torch.tensor(input_ids)
attention_masks = torch.tensor(attention_masks)
segment_ids = torch.tensor(segment_ids)

In [91]:
# Set save model to Evalue loop
save_model.eval();

In [92]:
# Get model predict result
with torch.no_grad():
        outputs = save_model(input_ids, token_type_ids=None,
        attention_mask=None,)
        # For eval mode, the first result of outputs is logits
        logits = outputs[0]

In [93]:
# Make logits into numpy type predict result
# The predict result contain each token's all tags predict result
predict_results = logits.detach().cpu().numpy()

In [94]:
predict_results.shape

(1, 45, 8)

In [95]:
from scipy.special import softmax

In [96]:
result_arrays_soft = softmax(predict_results[0])

In [97]:
result_arrays_soft[0]

array([1.1015027e-01, 2.8547707e-05, 9.8267097e-05, 2.9940004e-05,
       9.0841386e-05, 2.3996403e-05, 1.6651111e-05, 3.6762620e-05],
      dtype=float32)

In [98]:
result_array = result_arrays_soft

In [99]:
len(result_array),len(result_array[0])

(45, 8)

In [100]:
# Get each token final predict tag index result
result_list = np.argmax(result_array,axis=-1)

In [101]:
print(result_list)

[0 4 4 4 4 0 2 4 0 2 2 4 0 4 0 0 0 2 4 4 0 2 2 0 0 4 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0]


In [102]:
for i, mark in enumerate(attention_masks[0]):
    if mark>0:
        print("Token:%s"%(temp_token[i]))
        print("Tag:%s"%(result_list[i]))
        print("Predict_Tag:%s"%(tag2name[result_list[i]]))
        #print("Posibility:%f"%(result_array[i][result_list[i]]))
        print()

Token:[CLS]
Tag:0
Predict_Tag:B_INCLUSION

Token:Includes
Tag:4
Predict_Tag:O

Token:programs
Tag:4
Predict_Tag:O

Token:such
Tag:4
Predict_Tag:O

Token:as
Tag:4
Predict_Tag:O

Token:group
Tag:0
Predict_Tag:B_INCLUSION

Token:exercise
Tag:2
Predict_Tag:I_INCLUSION

Token:,
Tag:4
Predict_Tag:O

Token:adult
Tag:0
Predict_Tag:B_INCLUSION

Token:education
Tag:2
Predict_Tag:I_INCLUSION

Token:classes
Tag:2
Predict_Tag:I_INCLUSION

Token:,
Tag:4
Predict_Tag:O

Token:recreation
Tag:0
Predict_Tag:B_INCLUSION

Token:,
Tag:4
Predict_Tag:O

Token:nut
Tag:0
Predict_Tag:B_INCLUSION

Token:##rit
Tag:0
Predict_Tag:B_INCLUSION

Token:##ious
Tag:0
Predict_Tag:B_INCLUSION

Token:meals
Tag:2
Predict_Tag:I_INCLUSION

Token:,
Tag:4
Predict_Tag:O

Token:and
Tag:4
Predict_Tag:O

Token:social
Tag:0
Predict_Tag:B_INCLUSION

Token:work
Tag:2
Predict_Tag:I_INCLUSION

Token:services
Tag:2
Predict_Tag:I_INCLUSION

Token:.
Tag:0
Predict_Tag:B_INCLUSION

Token:[SEP]
Tag:0
Predict_Tag:B_INCLUSION



In [126]:
####Prediction#####
def bert_iob_predict(input_sentence):
    tokenized_texts = []
    temp_token = []
    temp_token.append('[CLS]')
    token_list = tokenizer.tokenize(input_sentence)
    for m,token in enumerate(token_list):
        inc_string = ""
        exc_string = ""
        temp_token.append(token)
    if len(temp_token) > max_len-1:
        temp_token= temp_token[:max_len-1]
    temp_token.append('[SEP]')
    tokenized_texts.append(temp_token)
    input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=max_len, dtype="long", truncating="post", padding="post")
    attention_masks = [[int(i>0) for i in ii] for ii in input_ids]
    segment_ids = [[0] * len(input_id) for input_id in input_ids]
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)
    segment_ids = torch.tensor(segment_ids)
    model.eval()
    with torch.no_grad():
       logits = model(b_input_ids, token_type_ids=None,
                       attention_mask=b_input_mask)
       logits = logits[0]
        
        # For eval mode, the first result of outputs is logits
    predict_results = logits.detach().cpu().numpy()
    # print(predict_results)
    result_arrays_soft = softmax(predict_results[0])
    #result_list = np.argmax(result_array,axis=-1)
    result_array = result_arrays_soft
    tags_pred = []
    token_pred = []
    for i, mark in enumerate(attention_masks[0]):
        if mark>0:
            # print("Token:%s"%(temp_token[i]))
            token_pred.append(temp_token[i])
            #print("Tag:%s"%(result_list[i]))
            # print("Predict_Tag:%s"%(tag2name[result_list[i]]))
            tags_pred.append(tag2name[result_list[i]])
            #print("Posibility:%f"%(result_array[i][result_list[i]]))
            # print()
    
    # tag2name[result_list[i]],
    return token_pred,tags_pred

In [104]:
bert_iob_predict("1) Coverage is available for up to one (1) visit per week for adult day center services.")

[[[ 5.84272480e+00 -1.84926176e+00 -1.44894814e+00 -1.00105929e+00
   -1.07198015e-01 -1.20104909e+00 -6.23666346e-01 -8.77204299e-01]
  [-5.95146537e-01 -1.27668142e+00 -6.88408494e-01 -1.18680918e+00
    1.94140124e+00 -1.19454074e+00  2.51594394e-01  1.37233949e+00]
  [-4.55421954e-01 -1.90294170e+00  2.25076243e-01 -1.51298165e+00
    1.10198963e+00 -1.59963655e+00  2.77515721e+00 -4.55001473e-01]
  [-6.34152532e-01 -1.25586891e+00  7.43330568e-02 -1.09170032e+00
    9.13674593e-01 -1.45986032e+00  2.65231776e+00  1.02290407e-01]
  [-1.72603679e+00 -1.38210356e+00  1.27796662e+00 -1.32638752e+00
    7.64644384e-01 -1.09266877e+00  2.90312958e+00  2.26431713e-01]
  [-2.24379945e+00 -1.62933481e+00  1.74564564e+00 -1.09507024e+00
    8.41328740e-01 -9.76149917e-01  2.94238305e+00 -6.20005786e-01]
  [-2.01485372e+00 -1.51138818e+00  2.33449054e+00 -1.46519458e+00
    1.56571376e+00 -1.39859390e+00  3.12493777e+00 -8.57932985e-01]
  [-1.85880125e+00 -1.25057399e+00 -1.48262167e+00 -1.6

(['[CLS]',
  '1',
  ')',
  'Cover',
  '##age',
  'is',
  'available',
  'for',
  'up',
  'to',
  'one',
  '(',
  '1',
  ')',
  'visit',
  'per',
  'week',
  'for',
  'adult',
  'day',
  'center',
  'services',
  '.',
  '[SEP]'],
 ['B_INCLUSION',
  'O',
  'O',
  'O',
  'O',
  'B_INCLUSION',
  'I_INCLUSION',
  'O',
  'B_INCLUSION',
  'I_INCLUSION',
  'I_INCLUSION',
  'O',
  'B_INCLUSION',
  'O',
  'B_INCLUSION',
  'B_INCLUSION',
  'B_INCLUSION',
  'I_INCLUSION',
  'O',
  'O',
  'B_INCLUSION',
  'I_INCLUSION',
  'I_INCLUSION',
  'B_INCLUSION'])

In [127]:
words,tags = bert_iob_predict(" - Laboratory and Radiology:  Testing or clinical studies of materials, fluids or tissues from patients, services include but are not limited to, the obtaining and testing of blood samples, histology, hematology, blood chemistry, pathology, histopathology, microbiology, and other diagnostic testing using physical specimens such as tissue, sputum, feces, urine or blood.")
print(words)
print(tags)
inclusions = []
exclusions = []
b_incl_index = []
b_excl_index = []
final_len = len(tags)

for i in range(len(tags)):
  print(tags[i])
  if tags[i] == "U_INCLUSION":
    inclusions.append(words[i])
  elif tags[i] == "U_EXCLUSION":
    exclusions.append(words[i])
  elif tags[i] == "B_INCLUSION":
    b_incl_index.append(i)
  elif tags[i] == "B_EXCLUSION":
    b_excl_index.append(i)
b_incl_index.append(final_len)
b_excl_index.append(final_len)

for i in range(len(b_incl_index)-1):
  incl_string = ""
  for j in range(b_incl_index[i],b_incl_index[i+1]):
    if tags[j]!="O":
      incl_string = incl_string + " " + str(words[j])
  inclusions.append(incl_string[1:])

for i in range(len(b_excl_index)-1):
  excl_string = ""
  for j in range(b_excl_index[i],b_excl_index[i+1]):
    if tags[j]!="O":
      excl_string = excl_string + " " + str(words[j])
  exclusions.append(excl_string[1:])

inclusions = list(set(inclusions) - set(['[CLS]','[SEP]']))
print(inclusions)
print(exclusions)

['[CLS]', '-', 'Laboratory', 'and', 'Radio', '##logy', ':', 'Testing', 'or', 'clinical', 'studies', 'of', 'materials', ',', 'fluids', 'or', 'tissues', 'from', 'patients', ',', 'services', 'include', 'but', 'are', 'not', 'limited', 'to', ',', 'the', 'obtaining', 'and', 'testing', 'of', 'blood', 'samples', ',', 'his', '##tology', ',', 'hem', '##ato', '##logy', ',', 'blood', '[SEP]']
['B_INCLUSION', 'O', 'O', 'O', 'O', 'B_INCLUSION', 'I_INCLUSION', 'O', 'B_INCLUSION', 'I_INCLUSION', 'I_INCLUSION', 'O', 'B_INCLUSION', 'O', 'B_INCLUSION', 'B_INCLUSION', 'B_INCLUSION', 'I_INCLUSION', 'O', 'O', 'B_INCLUSION', 'I_INCLUSION', 'I_INCLUSION', 'B_INCLUSION', 'B_INCLUSION', 'O', 'B_INCLUSION', 'B_INCLUSION', 'B_INCLUSION', 'B_INCLUSION', 'B_INCLUSION', 'B_INCLUSION', 'B_INCLUSION', 'B_INCLUSION', 'B_INCLUSION', 'B_INCLUSION', 'B_INCLUSION', 'B_INCLUSION', 'B_INCLUSION', 'B_INCLUSION', 'B_INCLUSION', 'B_INCLUSION', 'B_INCLUSION', 'B_INCLUSION', 'B_INCLUSION']
B_INCLUSION
O
O
O
O
B_INCLUSION
I_INCLUS

In [240]:
def prediction_func_bert(req_text):
  final_output = []
  req_text_df = df_data[df_data["requirement_text"] == req_text]
  sentences_df = []
  tags_org = []
  for i in range(len(req_text_df["sentence #"].unique())):
    temp_req_text_df = req_text_df[req_text_df['sentence #'] == req_text_df["sentence #"].unique()[i]]
    sentences_df.append(temp_req_text_df["pattern"].iloc[0])
    tags_org.append(list(temp_req_text_df["tags"]))

  # print(sentences_df)
  # print(tags_org)

  for sent in range(len(sentences_df)):
    words,tags = bert_iob_predict(sentences_df[sent])

    #### get inclusions, exclusions
    inclusions = []
    exclusions = []
    b_incl_index = []
    b_excl_index = []
    final_len = len(tags)

    for i in range(len(tags)):
      if tags[i] == "U_INCLUSION":
        inclusions.append(words[i])
      elif tags[i] == "U_EXCLUSION":
        exclusions.append(words[i])
      elif tags[i] == "B_INCLUSION":
        b_incl_index.append(i)
      elif tags[i] == "B_EXCLUSION":
        b_excl_index.append(i)
    b_incl_index.append(final_len)
    b_excl_index.append(final_len)

    for i in range(len(b_incl_index)-1):
      incl_string = ""
      for j in range(b_incl_index[i],b_incl_index[i+1]):
        if tags[j]!="O":
          incl_string = incl_string + " " + str(words[j])
      inclusions.append(incl_string[1:])

    for i in range(len(b_excl_index)-1):
      excl_string = ""
      for j in range(b_excl_index[i],b_excl_index[i+1]):
        if tags[j]!="O":
          excl_string = excl_string + " " + str(words[j])
      exclusions.append(excl_string[1:])

    inclusions = list(set(inclusions) - set(['[CLS]','[SEP]']))
    # print(inclusions)
    # print(exclusions)

    ###### get accuracy and f1

    tags_pred = tags
    tags_old = tags_org[sent]
    if len(tags_pred) < len(tags_old):
      diff = len(tags_old)-len(tags_pred)
      for i in range(diff):
        tags_pred.append('O')
    if len(tags_old) < len(tags_pred):
      diff = len(tags_pred)-len(tags_old)
      for i in range(diff):
        tags_old.append('O')

    # print("accuracy score: %f"%(accuracy_score(tags_pred, tags_old)))
    # print("f1 score: %f"%(f1_score([tags_pred], [tags_old])))

    temp_pred_json = {}
    temp_pred_json["Inclusions"] = ",".join(inclusions)
    temp_pred_json["Exclusions"] = ",".join(exclusions)
    temp_pred_json["Accuracy score"] = accuracy_score(tags_pred, tags_old)
    temp_pred_json["f1 score"] = f1_score([tags_pred], [tags_old])
    # print(temp_pred_json)
    final_output.append(temp_pred_json)
  print(final_output)
  return final_output


In [242]:
prediction_func_bert('Cardiac Rehabilitation Services')

[{'Inclusions': "condition,',nutrition,a supervised,pressure,factor,blood,;,includes counseling,information about the,and,on,information,high,such,controlling,smoking,risk,##ess,programs,; lifestyle and,as,##ation,c,,", 'Exclusions': '', 'Accuracy score': 0.28888888888888886, 'f1 score': 0.0}, {'Inclusions': 'services,of,rehabilitation,-,cardiac,each,factor,;,and,day,##an,furnished,Card,intensive cardiac rehabilitation,modification,the following,risk,Ph ##ys ##ici,all,must,exercise,##iac,rehabilitation programs,items,,,are', 'Exclusions': '', 'Accuracy score': 0.2222222222222222, 'f1 score': 0.0}, {'Inclusions': 'cardiac,programs,not qualify for,.,heart failure', 'Exclusions': '', 'Accuracy score': 0.17647058823529413, 'f1 score': 0.0}]


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_INCLUSION seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: I_INCLUSION seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_EXCLUSION seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: I_EXCLUSION seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


[{'Accuracy score': 0.28888888888888886,
  'Exclusions': '',
  'Inclusions': "condition,',nutrition,a supervised,pressure,factor,blood,;,includes counseling,information about the,and,on,information,high,such,controlling,smoking,risk,##ess,programs,; lifestyle and,as,##ation,c,,",
  'f1 score': 0.0},
 {'Accuracy score': 0.2222222222222222,
  'Exclusions': '',
  'Inclusions': 'services,of,rehabilitation,-,cardiac,each,factor,;,and,day,##an,furnished,Card,intensive cardiac rehabilitation,modification,the following,risk,Ph ##ys ##ici,all,must,exercise,##iac,rehabilitation programs,items,,,are',
  'f1 score': 0.0},
 {'Accuracy score': 0.17647058823529413,
  'Exclusions': '',
  'Inclusions': 'cardiac,programs,not qualify for,.,heart failure',
  'f1 score': 0.0}]

In [ ]:
!pip install gradio

In [243]:
import gradio as gr
benefit_iface = gr.Interface(fn=prediction_func_bert, inputs=gr.inputs.Textbox(default="Please enter the Benefit Text."), outputs="json")
benefit_iface.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)


INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.6p1)
INFO:paramiko.transport:Authentication (publickey) successful!


Running on External URL: https://53930.gradio.app
Interface loading below...


Tip: Add interpretation to your model by simply adding `interpretation="default"` to `Interface()`


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7861/',
 'https://53930.gradio.app')